In [ ]:
import io

import pandas as pd
import pygmt
import requests
from pygmt.params import Box, Position

params = {
    "format": "csv",
    "starttime": "2000-01-01",
    "endtime": "2025-10-30",
    "mindepth": 70,
    "minmagnitude": 5,
}
r = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query", params=params)
df_eqs = pd.read_csv(io.StringIO(r.text))
df_eqs = df_eqs[
    (df_eqs["longitude"] >= 91)
    & (df_eqs["longitude"] <= 134)
    & (df_eqs["latitude"] >= -22)
    & (df_eqs["latitude"] <= 18)
].sort_values(by="mag", ascending=False)

fig = pygmt.Figure()
fig.basemap(region=[91, 134, -22, 18], projection="M15c", frame=True)
fig.coast(land="gray95", shorelines="gray50")

# Plot epicenters with color (hypocentral depth) or size (magnitude)
pygmt.makecpt(cmap="SCM/navia", series=[0, 700], reverse=True, transparency=30)
fig.plot(
    x=df_eqs.longitude,
    y=df_eqs.latitude,
    style="c",
    size=0.005 * 2**df_eqs.mag,
    fill=df_eqs.depth,
    cmap=True,
    pen="gray10",
)
fig.colorbar(frame=["xaf+lHypocentral depth", "y+lkm"])
# Add legend for size-coding
legend = io.StringIO(
    "\n".join(f"S 0.4 c {0.005 * 2**mag:.2f} - 1p 1 M {mag}" for mag in [5, 6, 7])
)
fig.legend(
    spec=legend,
    position=Position("BR", offset=0.2),
    box=Box(fill="white", pen="black"),
    line_spacing=2,
)

# Add histogram for magnitude distribution
with fig.inset(
    position=Position("BL", offset=0.2),
    width=7,
    height=4,
    clearance=(1.3, 0.2, 1.1, 0.2),
    box=True,
):
    fig.histogram(
        region=[4.8, 9.2, 0, 0],
        projection="X?/?",
        frame=["WSrt", "xa1f0.2+lMagnitude", "yaf+lCounts"],
        data=df_eqs.mag,
        series=0.2,
        fill="darkgray",
        pen="lightgray",
    )

fig.show()
# fig.savefig("Fig4_PyGMT_pandas.png")